<div align="center">
<h1 style="color:#1565c0; font-size:2.8em; font-weight:bold; margin-bottom:0.2em; margin-top:0.5em;">Практична робота №2</h1>
<h3 style="color:#1976d2; margin-top:0; font-weight:normal;">Побудова та оцiнювання якостi
моделей класифiкацiї та регресiї засобами бiблiотеки <b>Scikit-Learn Python</b></h3>
<br>
<br>
<b>Варіант 12</b><br>
<b>Автори:</b><br>
Плахтій Злата, КА-32<br>
Пастушенко Максим, КА-33<br>
Романов Фелікс, КА-33
</div>

<b>Мета:</b> Метою роботи є побудова та оцiнювання якостi моделей:
- дерев рiшень,
- опорних векторiв,
- логiстичної регресiї,
- наївної баєсiвської моделi
для класифiкацiї та регресiї засобами бiблiотеки Scikit-Learn Python.

<b>Хід роботи:</b>
1. Підготувати дані.
2. Представити початкових даних графiчно.
3. Розбити данi на навчальний та валiдацiйний набори.
. Побудувати на навчальному наборi даних моделi класифiкацiї або регресiї заданi згiдно з варiантом.
4. Представити моделi графiчно (наприклад вивести частину дерева рiшень, побудувати лiнiю регресiї тощо).
5. Виконати прогнози на основi побудованих моделей.
6. Для кожної з моделей оцiнити, чи має мiсце перенавчання.
7. В задачах регресiї розрахувати для кожної моделi наступнi критерiї якостi, окремо на навчальнiй та валiдацiйнiй множинах:

    • коефiцiєнт детермiнацiї R2,
    
    • помилки RMSE, MAE та MAPE.

8. Спробувати виконати решiтчастий пошук (grid search) для пiдбору гiперпараметрiв моделей.
9. Зробити висновки про якiсть роботи моделей на дослiджених даних. На
основi критерiїв якостi спробувати обрати найкращу модель.
10. Навчити моделi на пiдмножинах навчальних даних. Оцiнити, наскiльки
розмiр навчальної множини впливає на якiсть моделi.
11. Кожний варiант мiстить два набори даних. Дослiдити обидва набори за
наведеними вище етапами.

<b>Завдання:</b><br>
Побудувати моделi регресiї на основi методу опорних векторiв:<br><br>
• Моделi нелiнiйної регресiї SVR(kernel="poly") з полiномiальним ядром. <br>
  Розглянути полiноми рiзного ступеня degree та рiзнi комбiнацiї гiперпараметрiв epsilon i C, наприклад: epsilon=0.1 i C=0.01; epsilon=0.1 i C=100.<br><br>
• Настроїти гiперпараметри epsilon i C, використовуючи решітчастий пошук.<br><br>
Початковi данi:<br>
(б) [avocado_prices.csv](https://www.kaggle.com/datasets/neuromusic/avocado-prices)
для якогось коміта

In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pathlib import Path

In [ ]:
# ------------------------------
# 1.1. Завантаження та огляд даних
# ------------------------------
# ------------- Налаштування -------------
FILE_PATH = 'data/avocado.csv'            # новий відносний шлях до файлу
TARGET = 'AveragePrice'
SAVE_PATH = 'data/avocado_prepared.csv'   # зберігати в тій самій папці (опціонально)

# Перевірка наявності файлу
if not Path(FILE_PATH).exists():
    raise FileNotFoundError(f"Файл не знайдено: {Path(FILE_PATH).resolve()}")

# Переконайся, що папка для збереження існує
out_dir = Path(SAVE_PATH).parent
out_dir.mkdir(parents=True, exist_ok=True)


# Завантажити дані та перевірити структуру 
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"Файл не знайдено: {FILE_PATH}")

df = pd.read_csv(FILE_PATH)

# Показати імена стовпців
print("Стовпці датасету:")
print(df.columns.tolist())
print("\n")

# Показати інформацію про датасет
print("Інформація про датасет:")
df.info()
print("\n")

# Показати описові статистики
print("Описові статистики:")
print(df.describe())
print("\n")


# Показати перші рядки датасету
print("Перші 5 рядків датасету:")
print(df.head())
print("\n")

Стовпці датасету:
['Unnamed: 0', 'Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year', 'region']


Інформація про датасет:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    18249 non-null  int64  
 1   Date          18249 non-null  object 
 2   AveragePrice  18249 non-null  float64
 3   Total Volume  18249 non-null  float64
 4   4046          18249 non-null  float64
 5   4225          18249 non-null  float64
 6   4770          18249 non-null  float64
 7   Total Bags    18249 non-null  float64
 8   Small Bags    18249 non-null  float64
 9   Large Bags    18249 non-null  float64
 10  XLarge Bags   18249 non-null  float64
 11  type          18249 non-null  object 
 12  year          18249 non-null  int64  
 13  region        18249 non-null  objec

In [ ]:
# ------------------------------
# 1.2. підготовка даних
# ------------------------------

# Видалити зайві стовпці 
# Часті «службові» назви: 'Unnamed: 0', 'Unnamed:0', 'X' або окремий стовпець year, що дублює дату
drop_candidates = ['Unnamed: 0', 'Unnamed:0', 'X', 'year']
drop_cols = [c for c in drop_candidates if c in df.columns]
if drop_cols:
    df.drop(columns=drop_cols, inplace=True)
    print('\nВидалені службові колонки:', drop_cols)

# Конвертувати Date у datetime 
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    # Якщо є пропуски у Date, краще видалити такі рядки
    n_date_na = df['Date'].isna().sum()
    if n_date_na:
        print(f"\nПримітка: знайдено {n_date_na} рядків з некоректною датою. Видаляю їх.")
        df = df.dropna(subset=['Date']).reset_index(drop=True)
    # Додаткові часові ознаки
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['day'] = df['Date'].dt.day
    print('\nСтворено часові ознаки: year, month, dayofweek, day')
else:
    print('\nУ датасеті відсутня колонка Date')

# Кодування категорій (type, region) 
cat_cols = [c for c in ['type', 'region'] if c in df.columns]
if cat_cols:
    # Використаємо get_dummies для простоти. drop_first=True щоб уникнути мультиколінеарності.
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    print(f"\nЗакодовані категоріальні колонки: {cat_cols} -> one-hot (drop_first=True)")
else:
    print('\nКатегоріальних колонок type/region не знайдено для кодування')


# Перейменувати колонки 4046, 4225, 4770 (PLU-коди) 
plu_codes = ['4046', '4225', '4770']
# Стовпці можуть бути рядками або числами. Зробимо універсальне перейменування.
rename_map = {}
for col in df.columns:
    col_str = str(col)
    if col_str in plu_codes:
        rename_map[col] = f'plu_{col_str}'
if rename_map:
    df.rename(columns=rename_map, inplace=True)
    print('\nПерейменовано PLU-колонки:', rename_map)
else:
    print('\nPLU-колонки 4046/4225/4770 не знайдено або вже перейменовано')

# Очищення даних: пропуски та дублікатні рядки 
# Пропуски
missing = df.isnull().sum()
missing = missing[missing > 0]
if not missing.empty:
    print('\nЗнайдені пропуски по колонках:')
    print(missing)
    # Для числових колонок заповнюємо медіаною, для категоріальних - модою
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # Не чіпаємо ціль, якщо у ній є пропуски - це краще розглянути окремо
    if TARGET in num_cols:
        num_cols.remove(TARGET)
    for c in missing.index:
        if c in num_cols:
            med = df[c].median()
            df[c].fillna(med, inplace=True)
            print(f"Заповнено пропуски в числовій колонці {c} медианою: {med}")
        else:
            mode = df[c].mode()
            if not mode.empty:
                df[c].fillna(mode.iloc[0], inplace=True)
                print(f"Заповнено пропуски в колонці {c} модою: {mode.iloc[0]}")
            else:
                df[c].fillna('', inplace=True)
                print(f"Заповнено пропуски в колонці {c} пустим рядком")
else:
    print('\nПропусків у датасеті не знайдено')

# Дублікатні рядки
n_dup = df.duplicated().sum()
if n_dup:
    print(f"\nЗнайдено {n_dup} дублікатів. Видаляю...")
    df.drop_duplicates(inplace=True)
else:
    print('\nДублікатів не знайдено')

# Масштабувати числові ознаки 
# Для SVR масштабування важливе. Масштабуємо всі числові колонки, крім цільової TARGET.
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if TARGET in num_cols:
    num_cols.remove(TARGET)

if num_cols:
    scaler = StandardScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols])
    print('\nМасштабовано числові ознаки (StandardScaler). Кількість колонок:', len(num_cols))
else:
    print('\nЧислових колонок для масштабування не знайдено')

# Збереження результату 
df.to_csv(SAVE_PATH, index=False)
print(f"\nПідготовка завершена. Збережено: {SAVE_PATH}")
print('\nРозміри після підготовки:', df.shape)
print(df.head())




Видалені службові колонки: ['year']

Створено часові ознаки: year, month, dayofweek, day

Категоріальних колонок type/region не знайдено для кодування

PLU-колонки 4046/4225/4770 не знайдено або вже перейменовано

Пропусків у датасеті не знайдено

Дублікатів не знайдено

Масштабовано числові ознаки (StandardScaler). Кількість колонок: 12

Підготовка завершена. Збережено: data/avocado_prepared.csv

Розміри після підготовки: (18249, 68)
        Date  AveragePrice  Total Volume  plu_4046  plu_4225  plu_4770  \
0 2015-12-27          1.33     -0.227716 -0.230816 -0.199902 -0.212091   
1 2015-12-20          1.35     -0.230427 -0.231103 -0.208054 -0.211997   
2 2015-12-13          0.93     -0.212085 -0.231007 -0.154478 -0.211325   
3 2015-12-06          1.08     -0.223444 -0.230741 -0.185350 -0.211864   
4 2015-11-29          1.28     -0.231538 -0.230891 -0.208719 -0.211834   

   Total Bags  Small Bags  Large Bags  XLarge Bags  ...  region_SouthCentral  \
0   -0.234170   -0.232647   -0.2223

In [ ]:

# ------------------------------
# 2.1. Візуалізація окремих стовпців
# ------------------------------
